In [ ]:
!pip install requests beautifulsoup4 pandas selenium

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

In [3]:
import csv
def write_recipes_to_csv(recipes, page_number):
    filename = f"recipes_page_{page_number}.csv"
    with open(filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(
            file,
            fieldnames=[
                "title",
                "url",
                "image_url",
                "author",
                "rating_percent",
                "cook_time",
            ],
        )
        writer.writeheader()
        writer.writerows(recipes)
    print(f"Data written to {filename}")

In [4]:
import requests
def fetch_recipe_data(page_number, timeout=60):
    url = f"https://api.food.com/services/mobile/fdc/search/sectionfront?pn={page_number}&recordType=Recipe&sortBy=trending&collectionId=17"

    try:
        response = requests.get(url, timeout=timeout)
        response.raise_for_status()  # Raise an error for bad responses (4xx and 5xx)
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        if "API rate limit" in str(e):  # Check if it's an API rate limit error
            raise
        return {}

In [1]:
import random
def scrape_recipes(category_urls, target_unique_titles=82212):
    unique_titles = set()  # Use a set to track unique titles
    recipes = []
    last_page_number = 17891

    for category_url in category_urls:
        page_number = 17891
        while len(unique_titles) < target_unique_titles:
            print(f"Fetching URL: {category_url}?page={page_number}")
            try:
                data = fetch_recipe_data(page_number)
            except requests.exceptions.ReadTimeout as e:
                print(f"Read timeout error: {e}")
                # Save state and return if read timeout error
                return recipes, last_page_number
            except Exception as e:
                print(f"Error fetching data: {e}")
                if "API rate limit" in str(e):
                    # Save state and return if rate limit error
                    print("API rate limit reached. Saving progress and exiting.")
                    return recipes, last_page_number

                # Random sleep to avoid hitting API rate limits
                time.sleep(random.uniform(1, 5))
                continue

            if not data.get("response") or not data.get("response").get("results"):
                print("No results found or no response, ending.")
                break

            for recipe in data.get("response").get("results", []):
                title = recipe.get("main_title", "").strip()
                if title in unique_titles:
                    continue  # Skip duplicate titles

                recipe_url = recipe.get("record_url")
                image_url = recipe.get("recipe_photo_url")
                author = recipe.get("main_username")
                rating_percent = recipe.get("main_rating")
                cook_time = recipe.get("recipe_totaltime")

                recipes.append(
                    {
                        "title": title,
                        "url": recipe_url,
                        "image_url": image_url,
                        "author": author,
                        "rating_percent": rating_percent,
                        "cook_time": cook_time,
                    }
                )

                unique_titles.add(title)  # Add title to the set

            print(f"Current count of unique recipes: {len(unique_titles)}")

            if len(unique_titles) >= target_unique_titles:
                break  # Stop if the target number of unique titles is reached

            # Write data to CSV every 100 pages
            if page_number % 100 == 0:
                write_recipes_to_csv(recipes, page_number)
                recipes = []  # Clear recipes list after saving to CSV

            print("Moving to next page")
            page_number += 1  # Move to the next page
            last_page_number = page_number

    # Write remaining data to CSV if needed
    if recipes:
        write_recipes_to_csv(recipes, last_page_number)

    return recipes, last_page_number

In [6]:
category_urls = [
    "https://www.food.com/recipe/all/trending",
    "https://www.food.com/recipe/all/quick-easy",
    "https://www.food.com/recipe/all/healthy",
    "https://www.food.com/recipe/all/editor-pick",
    "https://www.food.com/recipe/all/newest",
]

In [21]:
recipes = scrape_recipes(category_urls)

# Print or process the list of recipes
for recipe in recipes:
    print(recipe)

Fetching URL: https://www.food.com/recipe/all/trending?page=17891
Current count of unique recipes: 10
Moving to next page
Fetching URL: https://www.food.com/recipe/all/trending?page=17892
Current count of unique recipes: 20
Moving to next page
Fetching URL: https://www.food.com/recipe/all/trending?page=17893
Current count of unique recipes: 30
Moving to next page
Fetching URL: https://www.food.com/recipe/all/trending?page=17894
Current count of unique recipes: 40
Moving to next page
Fetching URL: https://www.food.com/recipe/all/trending?page=17895
Current count of unique recipes: 50
Moving to next page
Fetching URL: https://www.food.com/recipe/all/trending?page=17896
Current count of unique recipes: 60
Moving to next page
Fetching URL: https://www.food.com/recipe/all/trending?page=17897
Current count of unique recipes: 70
Moving to next page
Fetching URL: https://www.food.com/recipe/all/trending?page=17898
Current count of unique recipes: 80
Moving to next page
Fetching URL: https://ww

In [10]:
len(recipes)

2

In [92]:
# Extract titles
titles = [recipe["title"] for recipe in recipes]

# Find unique titles
unique_titles = set(titles)

# Number of unique titles
num_unique_titles = len(unique_titles)

print(f"Number of unique titles: {num_unique_titles}")

Number of unique titles: 2504


In [ ]:
import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Counter for errors
error_count = 0


def extract_recipe_details(recipe_url, cook_time, recipe_rating):
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # headless mode

    service = Service("/opt/homebrew/bin/chromedriver")
    driver = webdriver.Chrome(service=service, options=chrome_options)

    try:
        driver.get(recipe_url)

        # Wait for directions and ingredients to be present
        wait = WebDriverWait(driver, 3)
        directions_list = wait.until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".direction-list li"))
        )
        ingredient_elements = wait.until(
            EC.presence_of_all_elements_located(
                (By.CSS_SELECTOR, ".ingredient-list li")
            )
        )

        # Extract directions
        directions = "\n".join([li.text.strip() for li in directions_list])

        # Extract ingredients
        ingredients_dict = {}
        for element in ingredient_elements:
            quantity = element.find_element(
                By.CSS_SELECTOR, ".ingredient-quantity"
            ).text.strip()
            text = element.find_element(
                By.CSS_SELECTOR, ".ingredient-text"
            ).text.strip()
            ingredients_dict[quantity] = text

        return {
            "directions": directions,
            "ingredients": ingredients_dict,
            "cook_time": cook_time,
            "recipe_rating": recipe_rating,
        }

    except Exception as e:
        global error_count
        error_count += 1
        return {
            "directions": "N/A",
            "ingredients": {},
            "cook_time": cook_time,
            "recipe_rating": recipe_rating,
        }

    finally:
        driver.quit()


def process_csv_file(input_file, output_file):
    global error_count
    print(f"Processing file: {input_file}")

    # Read the CSV file
    recipes = pd.read_csv(input_file)

    # Prepare to collect results
    batch_size = 10
    all_recipes = []

    # Open the output file in append mode
    with open(output_file, "a") as f:
        # Write the header if the file is empty
        if os.stat(output_file).st_size == 0:
            pd.DataFrame(
                columns=[
                    "title",
                    "url",
                    "image_url",
                    "author",
                    "rating_percent",
                    "cook_time",
                    "directions",
                    "ingredients",
                ]
            ).to_csv(f, index=False)

    # Process recipes in batches
    for i, (_, recipe) in enumerate(recipes.iterrows()):
        try:
            details = extract_recipe_details(
                recipe["url"], recipe["cook_time"], recipe["rating_percent"]
            )
            all_recipes.append(
                {
                    "title": recipe["title"],
                    "url": recipe["url"],
                    "image_url": recipe["image_url"],
                    "author": recipe["author"],
                    "rating_percent": recipe["rating_percent"],
                    "cook_time": recipe["cook_time"],
                    "directions": details["directions"],
                    "ingredients": details["ingredients"],
                }
            )

            # Write to CSV every 10 recipes
            if (i + 1) % batch_size == 0:
                df = pd.DataFrame(all_recipes)
                df.to_csv(output_file, mode="a", header=False, index=False)
                all_recipes = []  # Reset list for the next batch
                print(f"Processed and saved batch of {batch_size} recipes.")

        except Exception:
            # Log general error message
            print(f"Failed to extract details for {recipe['url']}")

    # Process any remaining recipes that didn't fill a complete batch
    if all_recipes:
        df = pd.DataFrame(all_recipes)
        df.to_csv(output_file, mode="a", header=False, index=False)
        print(f"Processed and saved final batch of {len(all_recipes)} recipes.")

    # Print the total error count
    print(f"Total errors encountered: {error_count}")

    # Delete the input file once done
    os.remove(input_file)


input_directory = "/Users/snehsuresh/Desktop/Projects/recipe-recommender-system-pipeline/notebooks/data"
output_directory = "/Users/snehsuresh/Desktop/Projects/recipe-recommender-system-pipeline/notebooks/data/output"

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Get the list of CSV files and sort them
csv_files = [f for f in os.listdir(input_directory) if f.endswith(".csv")]
csv_files.sort()  # Sort filenames in ascending order

# Process each CSV file in the sorted order
for filename in csv_files:
    input_file = os.path.join(input_directory, filename)
    output_file = os.path.join(output_directory, f"processed_{filename}")
    process_csv_file(input_file, output_file)

In [14]:
import pandas as pd
from pathlib import Path

# Assuming all CSV files are in a directory named 'data'
csv_files = Path(
    "/Users/snehsuresh/Desktop/Projects/recipe-recommender-system-pipeline/notebooks/data/output"
).glob("*.csv")

In [15]:
df_list = [pd.read_csv(file, encoding="utf-8", delimiter=",") for file in csv_files]
recipes_df = pd.concat(df_list,  ignore_index=True)

In [ ]:
recipes_df

In [17]:
# Count non-NaN values in the 'directions' column
non_nan_directions_count = recipes_df["directions"].notna().sum()

# Print the count of non-NaN directions
print(f"Number of non-NaN values in 'directions': {non_nan_directions_count}")

Number of non-NaN values in 'directions': 53403


In [18]:
# Create a copy of the original DataFrame
df_copy = recipes_df.copy()

In [19]:
# Remove rows where 'directions' or 'ingredients' are NaN
df_copy = df_copy.dropna(subset=["directions", "ingredients"])

In [20]:
# Remove all rows with any NaN values
df_copy = df_copy.dropna()

In [23]:
df_copy

,title,url,image_url,author,rating_percent,cook_time,directions,ingredients,combined_text
58,Sparkling Honey Lemonade in Citrus-Salt Rimmed...,https://www.food.com/recipe/sparkling-honey-le...,https://img.sndimg.com/food/image/upload/v1/im...,SusieQusie,5.0,25,Combine lemon juice and honey in a saucepan.\n...,"{'1': 'lemon, zest of', '1⁄2': 'cup honey', '6...",Combine lemon juice and honey in a saucepan.\n...
60,Skillet Pizza Potatoes,https://www.food.com/recipe/skillet-pizza-pota...,https://img.sndimg.com/food/image/upload/v1/im...,ShaniRage,5.0,35,Cook sausage over medium high heat.\nStir in p...,"{'1': 'cup shredded Italian cheese blend', '1⁄...",Cook sausage over medium high heat.\nStir in p...
61,Black Forest Ice Cream Shake,https://www.food.com/recipe/black-forest-ice-c...,https://img.sndimg.com/food/image/upload/v1/im...,PanNan,5.0,15,Place the pint of chocolate ice cream on the c...,"{'1': 'cup milk', '1⁄2': 'cup chocolate chips'}",Place the pint of chocolate ice cream on the c...
62,Tomato Chokha,https://www.food.com/recipe/tomato-chokha-218896,https://img.sndimg.com/food/image/upload/v1/im...,WizzyTheStick,5.0,20,Preheat grill or broiler. Grill or broil tomat...,"{'4': 'large beefsteak tomatoes', '1': 'tables...",Preheat grill or broiler. Grill or broil tomat...
64,Linda's Mushroom and Squash Medley,https://www.food.com/recipe/lindas-mushroom-an...,https://img.sndimg.com/food/image/upload/v1/im...,Lindas Busy Kitchen,5.0,25,"Cut mushrooms, not too thin, about 4 slices to...","{'1': '(8 inch) summer squash, sliced', '1⁄2':...","Cut mushrooms, not too thin, about 4 slices to..."
...,...,...,...,...,...,...,...,...,...
64936,Big Berry Smoothie,https://www.food.com/recipe/big-berry-smoothie...,https://img.sndimg.com/food/image/upload/v1/im...,scarley,5.0,5,Add all the ingredients to a blender and pulse...,"{'2': 'cups ice', '1': 'cup plain yogurt', '1⁄...",Add all the ingredients to a blender and pulse...
64937,Farfalle With Tomato Herb Sauce,https://www.food.com/recipe/farfalle-with-toma...,https://img.sndimg.com/food/image/upload/v1/im...,OwlMonkey,5.0,45,"Heat the olive oil in a pan.\nAdd garlic, pepe...","{'1': 'tablespoon fresh parsley, chopped', '4'...","Heat the olive oil in a pan.\nAdd garlic, pepe..."
64938,Van Nuys (Aka Dirty Margarita),https://www.food.com/recipe/van-nuys-aka-dirty...,https://img.sndimg.com/food/image/upload/v1/im...,Boomette,5.0,2,Shake and strain into a thoroughly chilled mar...,"{'1 1⁄2': 'ounces tequila', '3⁄4': 'ounce Kahl...",Shake and strain into a thoroughly chilled mar...
64939,Crabmeat Hoezel,https://www.food.com/recipe/crabmeat-hoezel-38...,https://img.sndimg.com/food/image/upload/v1/im...,Laury,5.0,1,In a small bowl or jar combine tarragon vinega...,"{'1⁄4': 'cup extra virgin olive oil', '1⁄2': '...",In a small bowl or jar combine tarragon vinega...


In [22]:
# Combine directions and ingredients into a single text field
df_copy["combined_text"] = df_copy.apply(
    lambda row: f"{row['directions']} {' '.join(row['ingredients'].values()) if isinstance(row['ingredients'], dict) else ''}",
    axis=1,
)

In [24]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
import re

In [25]:
# Download necessary NLTK data
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/snehsuresh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/snehsuresh/nltk_data...


True

In [26]:
# Initialize stopwords and lemmatizer
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

In [27]:
def preprocess_text(text):
    text = re.sub(r"[^a-zA-Z\s]", "", text.lower())
    tokens = [
        lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words
    ]
    return " ".join(tokens)

In [28]:
# Convert ingredients dictionary to a string
def ingredients_to_string(ingredients_dict):
    return " ".join([f"{value}" for value in ingredients_dict.values()])

In [29]:
# Combine ingredients and directions into a single text field
df_copy["ingredients_str"] = df_copy["ingredients"].apply(
    lambda x: ingredients_to_string(eval(x)) if pd.notna(x) else ""
)

In [ ]:
df_copy["ingredients_str"]

In [31]:
df_copy["text"] = df_copy["ingredients_str"] + " " + df_copy["directions"]

In [33]:
df_copy["cleaned_text"] = df_copy["text"].apply(preprocess_text)

In [ ]:
df_copy["cleaned_text"]

In [35]:
# Create a document-term matrix for ingredients and directions combined
vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words="english")


In [36]:
vectorizer

CountVectorizer(max_df=0.95, min_df=2, stop_words='english')

In [38]:
dtm = vectorizer.fit_transform(df_copy["cleaned_text"])

In [40]:
# Fit LDA model
lda = LatentDirichletAllocation(
    n_components=10, random_state=42
)  # Adjust number of topics as needed
lda.fit(dtm)

LatentDirichletAllocation(random_state=42)

In [41]:
# Transform the original documents into topic distributions
topic_features = lda.transform(dtm)

In [42]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx}:")
        print(
            " ".join(
                [feature_names[i] for i in topic.argsort()[: -no_top_words - 1 : -1]]
            )
        )


# Display top 10 words in each topic
display_topics(lda, vectorizer.get_feature_names_out(), 10)

Topic 0:
cheese cup minute oven bake egg mixture dish baking ounce
Topic 1:
cup chocolate sugar sheet butter minute dough add baking cool
Topic 2:
chicken minute sauce place pepper tablespoon teaspoon grill pan breast
Topic 3:
dough flour bread minute potato cup bacon place oil salt
Topic 4:
cup teaspoon sugar cinnamon apple tablespoon mix minute brown ingredient
Topic 5:
pepper oil salt tablespoon olive cup add teaspoon garlic tomato
Topic 6:
cup juice cream ounce ice orange add lemon sugar glass
Topic 7:
add heat minute cook cup onion pepper stir salt teaspoon
Topic 8:
cup egg flour sugar pan minute add baking teaspoon bowl
Topic 9:
water minute potato place heat add boil remove use fish


In [43]:
from sklearn.metrics.pairwise import cosine_similarity

# Preprocess user input
user_input = (
    "flour baking powder cocoa powder sugar eggs"
)
user_input_processed = preprocess_text(user_input)
user_input_dtm = vectorizer.transform([user_input_processed])
user_input_topic_features = lda.transform(user_input_dtm)

In [44]:
# Compute similarity scores
similarities = cosine_similarity(user_input_topic_features, topic_features)

In [ ]:
# Get recommendations
top_indices = similarities[0].argsort()[-10:][::-1]  # Get top 10 recommended recipes
recommended_recipes = df_copy.iloc[top_indices]
print(recommended_recipes[["title", "directions"]])

In [47]:
from sklearn.decomposition import NMF

# Fit NMF model
nmf = NMF(n_components=10, random_state=42)
nmf.fit(dtm)

# Transform the document-term matrix into topic distributions
nmf_topic_features = nmf.transform(dtm)

In [48]:
import numpy as np
# Combine topic features from both models
combined_topic_features = np.hstack((topic_features, nmf_topic_features))

In [51]:
# Transform user input using both LDA and NMF
user_input_lda_features = lda.transform(user_input_dtm)
user_input_nmf_features = nmf.transform(user_input_dtm)

# Combine topic features from both models
user_input_combined_features = np.hstack(
    (user_input_lda_features, user_input_nmf_features)
)

In [52]:
similarities = cosine_similarity(user_input_combined_features, combined_topic_features)
# Get recommendations

In [53]:
top_indices = similarities[0].argsort()[-10:][::-1]  # Get top 10 recommended recipes
recommended_recipes = df_copy.iloc[top_indices]
print(recommended_recipes[["title", "directions"]])

                                 title  \
25966     Rose's World Famous Pancakes   
43521        Creamy Chocolate Frosting   
47579    Carrot Chocolate Chip Cookies   
32840             Dutch Baby - Adapted   
12507       Chocolate Sour Cream Icing   
12707  Betty Fords Buttermilk Pancakes   
51645          Easy Mix Chocolate Cake   
26402                     Yoghurt Cake   
1489                     Breakfast Bar   
59117         My Granny's Banana Bread   

                                              directions  
25966  Preheat griddle.\nBeat eggs and milk together....  
43521  Put all the ingredients into a large mixing bo...  
47579  un salted butter and flour and eggs and baking...  
32840  SIft together dry ingredients.\nStir in wet in...  
12507  Cream butter and sugar together.\nAdd remainin...  
12707                                mix together\ncook.  
51645  Place all ingredients in a large bowl and mix ...  
26402  Beat all ingredients together for 3 minutes.\n...  
1489  